# Himmelblau Benchmark
## Imports

In [1]:
from bofire.benchmarks.single import Hartmann
from bofire.benchmarks.benchmark import run
from bofire.data_models.strategies.api import SoboStrategy, PolytopeSampler
from bofire.data_models.acquisition_functions.api import qEI
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
from functools import partial
import pandas as pd
import os

SMOKE_TEST = os.environ.get("SMOKE_TEST")

/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Random Optimization

In [13]:
def sample(domain):
    datamodel = PolytopeSampler(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled

def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()

random_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=PolytopeSampler,
    n_iterations= 50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best -0.965: 100%|██████████| 50/50 [00:00<00:00, 101.10it/s]


## SOBO (GPEI) Optimization

In [3]:
from functools import partial

bo_results = run(
    Hartmann(dim=6, allowed_k=4),
    strategy_factory=partial(SoboStrategy, acquisition_function=qEI(), num_raw_samples=512, num_restarts=24),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best -2.030:  24%|██▍       | 12/50 [01:15<04:03,  6.40s/it]/Users/j30607/sandbox/botorch/botorch/optim/optimize.py:366: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 8 and message Positive directional derivative for linesearch.'), UserWarning('SLSQP failed to converge to a solution the satisfies the non-linear constraints. Returning the feasible starting point.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)
run 00 with current best -2.233:  26%|██▌       | 13/50 [01:29<05:13,  8.48s/it]/Users/j30607/sandbox/botorch/botorch/optim/optimize.py:366: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 5 and message Singular matrix E in LSQ subproblem.')]
Tryi